In [1]:
import torch
print(torch.cuda.is_available())  # Debe imprimir True si la GPU está funcionando
print(torch.cuda.get_device_name(0))  # Nombre de la GPU

ModuleNotFoundError: No module named 'torch'

El anterior es un modelo más grande

MINIPRUEBA

In [3]:
!pip install pdf2image -q
!pip install pdfminer -q
!pip install pdfminer.six -q
!pip install openai -q
!pip install scikit-learn -q
!pip install rich -q
!pip install tqdm -q
!pip install pandas -q

In [ ]:
!pip install transformers==4.41.0 sentence-transformers
!pip install faiss-cpu
!pip install ollama

In [1]:
from pdf2image import convert_from_path
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
from pdfminer.high_level import extract_text
import base64
import io
import os
import concurrent.futures
from tqdm import tqdm
import re
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
from rich import print
from ast import literal_eval

In [2]:
def convert_doc_to_images(path):
    images = convert_from_path(path)
    return images

def extract_text_from_doc(path):
    text = extract_text(path)
    return text

In [3]:
file_path = "8VM3BlueGIS.pdf"

images = convert_doc_to_images(file_path)
text = extract_text_from_doc(file_path)

In [4]:
text

'8VM3 Blue GISTM\n\nComing soon to our Blue portfolio:\nCompact gas-insulated switchgear up to 72.5 kV\nClean air switchgear with Zero CO2e emissions\n\nsiemens-energy.com/blue-products\n\n\x0c8VM3 Blue GISTM\n\nVacuum technology\n\nClean air\n\nBlue\n\n+\n\nHigher voltages with \nZero GWP\n\nRenewable energy sources are key to the \ndecarbonization of the energy sector – and \nas the demand for power increases, a new \ngeneration of wind power plants is needed \nto handle the higher voltage created.\nAt the same time as increasing capacity, it’s \nessential that we achieve Zero Global \nWarming Potential (GWP) in the transmis-\nsion of power as well as in its generation.\n\nMost high-voltage electrical switchgear still \nuses SF6 gas for insulation – but this is \n24,300 times more climate-hostile than CO2\nand stays in the atmosphere for up to 1,000 \nyears. Recognizing the harmfulness of SF6\nand other F-gases, the EU has legislated to \nrestrict their use and allows only GWP < 1 \n

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Modelo pequeño para embeddings
modelo_embeddings = SentenceTransformer("all-MiniLM-L6-v2")

# Dividir el texto en fragmentos
fragmentos = text.split("\n")  # Separar por oraciones

# Convertir cada fragmento en embeddings
embeddings = modelo_embeddings.encode(fragmentos)

# Crear base de datos FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


c:\Users\ra270\UPC FIB GIA\3º Any\Q2\PIA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ra270\UPC FIB GIA\3º Any\Q2\PIA\.venv\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
fragmentos

['8VM3 Blue GISTM',
 '',
 'Coming soon to our Blue portfolio:',
 'Compact gas-insulated switchgear up to 72.5 kV',
 'Clean air switchgear with Zero CO2e emissions',
 '',
 'siemens-energy.com/blue-products',
 '',
 '\x0c8VM3 Blue GISTM',
 '',
 'Vacuum technology',
 '',
 'Clean air',
 '',
 'Blue',
 '',
 '+',
 '',
 'Higher voltages with ',
 'Zero GWP',
 '',
 'Renewable energy sources are key to the ',
 'decarbonization of the energy sector – and ',
 'as the demand for power increases, a new ',
 'generation of wind power plants is needed ',
 'to handle the higher voltage created.',
 'At the same time as increasing capacity, it’s ',
 'essential that we achieve Zero Global ',
 'Warming Potential (GWP) in the transmis-',
 'sion of power as well as in its generation.',
 '',
 'Most high-voltage electrical switchgear still ',
 'uses SF6 gas for insulation – but this is ',
 '24,300 times more climate-hostile than CO2',
 'and stays in the atmosphere for up to 1,000 ',
 'years. Recognizing the harmf

In [ ]:
!ollama hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
!ollama pull hf.co/bartowski/Humanish-LLama3-8B-Instruct-GGUF

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏    0 B/4.9 GB                  pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏    0 B/4.9 GB                  pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏    0 B/4.9 GB                  pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏    0 B/4.9 GB                  pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏    0 B/4.9 GB                  pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏ 8.2 KB/4.9 GB                  pulling manifest 
pulling 6061136cfdfd...   0% ▕                ▏ 193 KB/4.9 GB                  pulling manifest 
pulling 6061136cfd

In [12]:
import ollama
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings']
  VECTOR_DB.append((chunk, embedding))

In [ ]:
for i, chunk in enumerate(fragmentos):
  add_chunk_to_database(chunk)
print(f'Added chunks {len(fragmentos)} to the database')

In [ ]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [33]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]

In [34]:
When is the first major inspection?

Object `inspection` not found.


In [44]:
!pip install pdfplumber

  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.5-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl (3.0 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20250324
    Uninstalling pdfminer.six-20250324:
      Successfully uninstalled pdfminer.six-20250324


In [43]:
#input_query = input('Ask me a question: ')
input_query = 'When is the first major inspection?'
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''

IndexError: list index out of range

In [22]:
print(input_query)

When is the first major inspection?

In [ ]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)